## **Create New BQ Dataset**

In [1]:
dataset_id = "datamart"

In [2]:
!bq --location=US mk --dataset {dataset_id}

Dataset 'authentic-light-303018:datamart' successfully created.


## **Create Modeled Tables and Convert Data Types**

#### Create NameBasics table and convert the birthYear and deathYear fields to INT64

In [35]:
%%bigquery
CREATE TABLE datamart.NameBasics AS 
SELECT nconst, primaryName, CAST(birthYear as INT64) as birthYear, CAST(deathYear as INT64) as deathYear, primaryProfession, knownForTitles FROM imdb_staging.NameBasics WHERE birthYear != "\\N" AND deathYear != "\\N"
UNION DISTINCT
SELECT nconst, primaryName, NULL, NULL, primaryProfession, knownForTitles FROM imdb_staging.NameBasics WHERE birthYear = "\\N" and deathYear = "\\N"
UNION DISTINCT
SELECT nconst, primaryName, CAST(birthYear as INT64) as birthYear, NULL, primaryProfession, knownForTitles FROM imdb_staging.NameBasics WHERE birthYear != "\\N" AND deathYear = "\\N"
UNION DISTINCT
SELECT nconst, primaryName, NULL, CAST(deathYear as INT64) as deathYear, primaryProfession, knownForTitles FROM imdb_staging.NameBasics WHERE birthYear = "\\N" AND deathYear != "\\N"

""


**Check to see if everything was added correctly**

In [39]:
%%bigquery
SELECT * FROM datamart.NameBasics LIMIT 5

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm12396807,Sigmund Jones,NaN,NaN,stunts,tt0228690
1,nm9903591,Mark Howard,NaN,NaN,"cinematographer,editor",tt8501648
2,nm1704605,V. Gorelova,NaN,NaN,"editorial_department,set_decorator","tt0270784,tt0106049"
3,nm1880314,Martín Wain,NaN,NaN,"director,writer,producer","tt0781391,tt0830800,tt0780014,tt0437444"
4,nm1881958,Pamela Mattioli,NaN,NaN,"actress,casting_department","tt1213825,tt0942886,tt1512273,tt0454846"


In [43]:
%%bigquery 
SELECT * FROM datamart.NameBasics WHERE birthYear IS NOT null AND deathYear IS null LIMIT 5

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0122839,R.J. Burns,1950,NaN,"actor,camera_department,casting_department","tt0120786,tt0118928,tt0145971,tt0383478"
1,nm0005891,Tim Suhrstedt,1948,NaN,"cinematographer,camera_department","tt0090887,tt0096928,tt0095690,tt0387808"
2,nm1456336,Gudrun Ziegler,1942,NaN,"producer,director,writer","tt0893502,tt0382491,tt2143588,tt0326706"
3,nm0374287,Milan Hein,1946,NaN,"actor,writer","tt0905992,tt0395962,tt0078054,tt0074846"
4,nm1538239,Brea Bee,1975,NaN,"actress,producer","tt1045658,tt0413573,tt0056758,tt8619822"


In [44]:
%%bigquery 
SELECT * FROM datamart.NameBasics WHERE birthYear IS null AND deathYear IS NOT null LIMIT 5

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm7888790,Nikolai Obukhovich,NaN,2016,"director,writer,cinematographer","tt10006182,tt5419750"
1,nm0662561,Ray Parker,NaN,2014,"miscellaneous,writer,script_department","tt0169492,tt0147747,tt0078581,tt0072499"
2,nm8468956,Saeed Karimian,NaN,2017,"producer,director","tt6127364,tt6875506,tt7666988,tt6126284"
3,nm0761099,Emilio Sancho,NaN,1967,"miscellaneous,actor","tt0055310,tt0048066,tt0045896,tt0058943"
4,nm0913750,Julian Wastall,NaN,1994,"composer,soundtrack,music_department","tt0100024,tt0105977,tt0096343,tt1963768"


#### **The primary key for this table is nconst. check for any records with the same primary key**

In [33]:
%%bigquery
SELECT count(*) as record_count FROM datamart.NameBasics GROUP BY nconst HAVING count(*) > 1

,record_count


#### **CREATE TitleBasics table by joining imdb_staging.TitleBasics with imdb_staging.TitleRatings**

**We need to create a temporary table because we will have to later convert the data types of this table**

In [79]:
%%bigquery
CREATE TABLE datamart.temp AS
SELECT t.tconst, t.titleType, t.primaryTitle, t.originalTitle, t.isAdult, t.startYear, t.endYear, t.runtimeMinutes, t.genres, r.averageRating, r.numVotes
FROM imdb_staging.TitleBasics as t LEFT JOIN imdb_staging.TitleRatings as r ON t.tconst = r.tconst

""


#### **Convert isAdult to BOOL, startYear to INT, endYear to INT, and runtimeMinutes to INT**

**Check if isAdult, startYear, endYear, and runtimeMinutes have any null values**

In [52]:
%%bigquery
SELECT * from imdb_staging.TitleBasics where isAdult = "\\N"

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [54]:
%%bigquery
SELECT * from imdb_staging.TitleBasics where startYear = "\\N" LIMIT 10

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt4271080,tvEpisode,Episode #1.84,Episode #1.84,0,\N,\N,\N,"Crime,Drama,Horror"
1,tt4282008,tvEpisode,Episode #1.41,Episode #1.41,0,\N,\N,\N,"Comedy,Family"
2,tt4301058,tvEpisode,Seres de Luz,Seres de Luz,0,\N,\N,\N,"Comedy,Music,Romance"
3,tt4302178,tvEpisode,Episode #1.5,Episode #1.5,0,\N,\N,\N,"Crime,Mystery"
4,tt4315112,movie,Esperanza Rising,Esperanza Rising,0,\N,\N,\N,Musical
5,tt4315848,tvEpisode,Episode #3.16,Episode #3.16,0,\N,\N,\N,"Action,Crime,Drama"
6,tt4318276,tvEpisode,Episode #1.3,Episode #1.3,0,\N,\N,\N,"Biography,Drama"
7,tt4348848,tvEpisode,Episode #1.3,Episode #1.3,0,\N,\N,\N,"Drama,Romance,War"
8,tt4353748,tvEpisode,Episode #1.3,Episode #1.3,0,\N,\N,\N,Action
9,tt4373124,tvEpisode,Episode #5.22,Episode #5.22,0,\N,\N,46,"Crime,Mystery"


**Based on this query, startYear, endYear, and runtimeMinutes all have NULL values**

In [109]:
%%bigquery
CREATE TABLE datamart.TitleBasics AS
SELECT tconst, titleType, primaryTitle, originalTitle, FALSE as isAdult, CAST(startYear as INT64) as startYear, CAST(endYear as INT64) as endYear, CAST(runtimeMinutes as INT64) as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear != "\\N" AND endYear != "\\N" AND runtimeMinutes != "\\N" AND isAdult = "0"

""


In [110]:
%%bigquery
INSERT INTO datamart.TitleBasics
SELECT tconst, titleType, primaryTitle, originalTitle, TRUE as isAdult, CAST(startYear as INT64) as startYear, CAST(endYear as INT64) as endYear, CAST(runtimeMinutes as INT64) as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear != "\\N" AND endYear != "\\N" AND runtimeMinutes != "\\N" AND isAdult = "1"

""


In [111]:
%%bigquery
INSERT INTO datamart.TitleBasics
SELECT tconst, titleType, primaryTitle, originalTitle, FALSE as isAdult, NULL as startYear, CAST(endYear as INT64) as endYear, CAST(runtimeMinutes as INT64) as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear = "\\N" AND endYear != "\\N" AND runtimeMinutes != "\\N" AND isAdult = "0"

""


In [112]:
%%bigquery
INSERT INTO datamart.TitleBasics
SELECT tconst, titleType, primaryTitle, originalTitle, TRUE as isAdult, NULL as startYear, CAST(endYear as INT64) as endYear, CAST(runtimeMinutes as INT64) as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear = "\\N" AND endYear != "\\N" AND runtimeMinutes != "\\N" AND isAdult = "1"

""


In [113]:
%%bigquery
INSERT INTO datamart.TitleBasics
SELECT tconst, titleType, primaryTitle, originalTitle, FALSE as isAdult, CAST(startYear as INT64) as startYear, NULL as endYear, CAST(runtimeMinutes as INT64) as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear != "\\N" AND endYear = "\\N" AND runtimeMinutes != "\\N" AND isAdult = "0"

""


In [114]:
%%bigquery
INSERT INTO datamart.TitleBasics
SELECT tconst, titleType, primaryTitle, originalTitle, TRUE as isAdult, CAST(startYear as INT64) as startYear, NULL as endYear, CAST(runtimeMinutes as INT64) as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear != "\\N" AND endYear = "\\N" AND runtimeMinutes != "\\N" AND isAdult = "1"

""


In [115]:
%%bigquery
INSERT INTO datamart.TitleBasics
SELECT tconst, titleType, primaryTitle, originalTitle, FALSE as isAdult, CAST(startYear as INT64) as startYear, CAST(endYear as INT64) as endYear, NULL as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear != "\\N" AND endYear != "\\N" AND runtimeMinutes = "\\N" AND isAdult = "0"

""


In [116]:
%%bigquery
INSERT INTO datamart.TitleBasics
SELECT tconst, titleType, primaryTitle, originalTitle,TRUE as isAdult, CAST(startYear as INT64) as startYear, CAST(endYear as INT64) as endYear, NULL as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear != "\\N" AND endYear != "\\N" AND runtimeMinutes = "\\N" AND isAdult = "1"

""


In [117]:
%%bigquery
INSERT INTO datamart.TitleBasics
SELECT tconst, titleType, primaryTitle, originalTitle, FALSE as isAdult, NULL as startYear, NULL as endYear, CAST(runtimeMinutes as INT64) as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear = "\\N" AND endYear = "\\N" AND runtimeMinutes != "\\N" AND isAdult = "0"

""


In [118]:
%%bigquery
INSERT INTO datamart.TitleBasics
SELECT tconst, titleType, primaryTitle, originalTitle, TRUE as isAdult, NULL as startYear, NULL as endYear, CAST(runtimeMinutes as INT64) as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear = "\\N" AND endYear = "\\N" AND runtimeMinutes != "\\N" AND isAdult = "1"

""


In [119]:
%%bigquery
INSERT INTO datamart.TitleBasics
SELECT tconst, titleType, primaryTitle, originalTitle, FALSE as isAdult, NULL as startYear, CAST(endYear as INT64) as endYear, NULL as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear = "\\N" AND endYear != "\\N" AND runtimeMinutes = "\\N" AND isAdult = "0"

""


In [120]:
%%bigquery
INSERT INTO datamart.TitleBasics
SELECT tconst, titleType, primaryTitle, originalTitle, TRUE as isAdult, NULL as startYear, CAST(endYear as INT64) as endYear, NULL as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear = "\\N" AND endYear != "\\N" AND runtimeMinutes = "\\N" AND isAdult = "1"

""


In [121]:
%%bigquery
INSERT INTO datamart.TitleBasics
SELECT tconst, titleType, primaryTitle, originalTitle, FALSE as isAdult, CAST(startYear as INT64) as startYear, NULL as endYear, NULL as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear != "\\N" AND endYear = "\\N" AND runtimeMinutes = "\\N" AND isAdult = "0"

""


In [122]:
%%bigquery
INSERT INTO datamart.TitleBasics
SELECT tconst, titleType, primaryTitle, originalTitle, TRUE as isAdult, CAST(startYear as INT64) as startYear, NULL as endYear, NULL as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear != "\\N" AND endYear = "\\N" AND runtimeMinutes = "\\N" AND isAdult = "1"

""


In [123]:
%%bigquery
INSERT INTO datamart.TitleBasics
SELECT tconst, titleType, primaryTitle, originalTitle, FALSE as isAdult, NULL as startYear, NULL as endYear, NULL as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear = "\\N" AND endYear = "\\N" AND runtimeMinutes = "\\N" AND isAdult = "0"

""


In [124]:
%%bigquery
INSERT INTO datamart.TitleBasics
SELECT tconst, titleType, primaryTitle, originalTitle, TRUE as isAdult, NULL as startYear, NULL as endYear, NULL as runtimeMinutes, genres, averageRating, numVotes
FROM datamart.temp
WHERE startYear = "\\N" AND endYear = "\\N" AND runtimeMinutes = "\\N" AND isAdult = "1"

""


**Check if everything loaded correctly**

In [129]:
%%bigquery
SELECT * FROM datamart.TitleBasics WHERE isAdult IS FALSE LIMIT 5

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0262957,tvMiniSeries,The Face of Tutankhamun,The Face of Tutankhamun,False,1992,NaN,197,"Documentary,History",8.2,27
1,tt3995480,tvMiniSeries,Intrusion,Intrusion,False,2015,NaN,140,"Drama,Mystery,Thriller",7.0,60
2,tt5904812,tvMiniSeries,The Indian Wars: A Change of Worlds,The Indian Wars: A Change of Worlds,False,2016,NaN,357,"Biography,Documentary,History",5.1,36
3,tt11361808,tvSpecial,Royal Rumble,Royal Rumble,False,2020,NaN,252,"Action,Sport",7.5,357
4,tt0196055,tvSpecial,Robin Williams: An Evening at the Met,Robin Williams: Live at the Met,False,1986,NaN,65,"Comedy,Documentary",8.4,1357


#### **The primary key for this table is tconst. check for any records with the same primary key**

In [34]:
%%bigquery
SELECT count(*) as record_count FROM datamart.TitleBasics GROUP BY tconst HAVING count(*) > 1

,record_count


#### **Create the TitleAkas table and convert ordering to INT and isOriginalTitle to BOOL**

In [132]:
%%bigquery
CREATE TABLE datamart.TitleAkas AS
SELECT titleId, CAST(ordering as INT64) as ordering, title, region, language, types, attributes, FALSE as isOriginalTitle FROM imdb_staging.TitleAkas WHERE isOriginalTitle = "0"

""


In [135]:
%%bigquery
INSERT INTO datamart.TitleAkas
SELECT titleId, CAST(ordering as INT64) as ordering, title, region, language, types, attributes, TRUE as isOriginalTitle FROM imdb_staging.TitleAkas WHERE isOriginalTitle = "1"

""


In [144]:
%%bigquery
INSERT INTO datamart.TitleAkas
SELECT titleId, CAST(ordering as INT64) as ordering, title, region, language, types, attributes, NULL as isOriginalTitle FROM imdb_staging.TitleAkas WHERE isOriginalTitle = "\\N"

""


**Remove orphaned records**

In [8]:
%%bigquery
DELETE FROM datamart.TitleAkas t
WHERE NOT EXISTS(SELECT * FROM datamart.TitleBasics b WHERE t.titleId = b.tconst)

""


**Check if the data is loaded correctly**

In [145]:
%%bigquery
SELECT * FROM datamart.TitleAkas WHERE isOriginalTitle IS TRUE LIMIT 5

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt7040678,3,Namoro À Distância,BR,\N,original,\N,True
1,tt4537986,2,Zhui bu,CN,cmn,original,\N,True
2,tt9317864,1,Min sang til far,DK,da,original,\N,True
3,tt2838166,5,Who is coming,GB,\N,original,\N,True
4,tt4089772,2,Si tou hang jeh,HK,yue,original,\N,True


In [148]:
%%bigquery
SELECT * FROM datamart.TitleAkas LIMIT 5

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt1841747,1,Moments of Fiction,AE,\N,\N,\N,None
1,tt1993290,1,Briquettes Into Bridges,AF,\N,\N,\N,None
2,tt2460072,1,Sobre la distancia,AR,\N,\N,\N,None
3,tt2363209,1,Guiso de iguana,AR,\N,\N,\N,None
4,tt5185838,1,La guerra por el metro cuadrado,AR,\N,\N,\N,None


#### **The primary key for this table is the combination of titleId and ordering. check for any records with the same primary key**

In [46]:
%%bigquery
SELECT count(*) as record_count FROM datamart.TitleAkas GROUP BY titleId, ordering HAVING count(*) > 1

,record_count


#### **Create TitleEpisode table and convert seasonNumber and episodeNumber to INT**

In [150]:
%%bigquery
CREATE TABLE datamart.TitleEpisode AS
SELECT tconst, parentTconst, CAST(seasonNumber as INT64) as seasonNumber, CAST(episodeNumber as INT64) as episodeNumber FROM imdb_staging.TitleEpisode WHERE seasonNumber != "\\N" AND episodeNumber != "\\N"

""


In [151]:
%%bigquery
INSERT INTO datamart.TitleEpisode
SELECT tconst, parentTconst, NULL as seasonNumber, CAST(episodeNumber as INT64) as episodeNumber FROM imdb_staging.TitleEpisode WHERE seasonNumber = "\\N" AND episodeNumber != "\\N"

""


In [152]:
%%bigquery
INSERT INTO datamart.TitleEpisode
SELECT tconst, parentTconst, CAST(seasonNumber as INT64) as seasonNumber, NULL as episodeNumber FROM imdb_staging.TitleEpisode WHERE seasonNumber != "\\N" AND episodeNumber = "\\N"

""


In [153]:
%%bigquery
INSERT INTO datamart.TitleEpisode
SELECT tconst, parentTconst, NULL as seasonNumber, NULL as episodeNumber FROM imdb_staging.TitleEpisode WHERE seasonNumber = "\\N" AND episodeNumber = "\\N"

""


**remove orphaned records**

In [16]:
%%bigquery
DELETE FROM datamart.TitleEpisode e
WHERE NOT EXISTS(SELECT * FROM datamart.TitleBasics b WHERE e.tconst = b.tconst)

""


**check**

In [154]:
%%bigquery
SELECT * FROM datamart.TitleEpisode LIMIT 5

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0076269,tt0159876,NaN,NaN
1,tt0093923,tt0974356,NaN,NaN
2,tt0102145,tt2138025,NaN,NaN
3,tt0116907,tt0159876,NaN,NaN
4,tt0154507,tt0386922,NaN,NaN


In [155]:
%%bigquery
SELECT * FROM datamart.TitleEpisode WHERE seasonNumber IS NOT NULL AND episodeNumber IS NOT NULL LIMIT 5

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0509203,tt0065272,35,58
1,tt0511562,tt0049993,15,40
2,tt0524692,tt0084987,18,64
3,tt0524891,tt0084987,20,73
4,tt0525109,tt0084987,16,81


#### **The primary key for this table is tconst. check for any records with the same primary key**

In [48]:
%%bigquery
SELECT count(*) as record_count FROM datamart.TitleEpisode GROUP BY tconst HAVING count(*) > 1

,record_count


#### **Create the HasDirector table and join with TitleBasics to add the primarytitle and remove any orphaned records**

In [2]:
%%bigquery
CREATE TABLE datamart.HasDirector AS
SELECT c.tconst, t.primaryTitle, c.directors 
FROM imdb_staging.TitleCrew c
JOIN imdb_staging.TitleBasics t
ON t.tconst = c.tconst

""


**Check**

In [6]:
%%bigquery
SELECT * FROM datamart.HasDirector ORDER BY primaryTitle LIMIT 5

,tconst,primaryTitle,directors
0,tt2386381,!Next?,nm4133405
1,tt2071912,!Que ve el Bisbe!,nm1355949
2,tt1699720,!Women Art Revolution,nm0380961
3,tt3690350,"""#RichKids of Beverly Hills"" Star EJ Johnson/C...",nm2083822
4,tt3698022,"""#selfie"" by The Chainsmokers",nm4386928


#### **The primary key for this table is tconst. check for any records with the same primary key**

In [49]:
%%bigquery
SELECT count(*) as record_count FROM datamart.HasDirector GROUP BY tconst HAVING count(*) > 1

,record_count


#### **Create the HasWriter table and join with TitleBasics to add the primarytitle and remove any orphaned records**

In [4]:
%%bigquery
CREATE TABLE datamart.HasWriter AS
SELECT c.tconst, t.primaryTitle, c.writers 
FROM imdb_staging.TitleCrew c
JOIN imdb_staging.TitleBasics t
ON t.tconst = c.tconst

""


**Check**

In [7]:
%%bigquery
SELECT * FROM datamart.HasWriter ORDER BY primaryTitle LIMIT 5

,tconst,primaryTitle,writers
0,tt2386381,!Next?,\N
1,tt2071912,!Que ve el Bisbe!,"nm1313785,nm4756203"
2,tt1699720,!Women Art Revolution,\N
3,tt3690350,"""#RichKids of Beverly Hills"" Star EJ Johnson/C...",\N
4,tt3698022,"""#selfie"" by The Chainsmokers",nm4386928


#### **The primary key for this table is tconst. check for any records with the same primary key**

In [51]:
%%bigquery
SELECT count(*) as record_count FROM datamart.HasWriter GROUP BY tconst HAVING count(*) > 1

,record_count


#### **Create the TitlePrincipals table and join with TitleBasics and NameBasics to add the primaryTitle, primaryName, and remove orphaned records**

In [90]:
%%bigquery
CREATE TABLE datamart.TitlePrincipals AS
SELECT p.tconst, t.primaryTitle, p.nconst, n.primaryName, CAST(p.ordering as INT64) as ordering, p.category, p.job, p.characters
FROM imdb_staging.TitlePrincipals p
JOIN imdb_staging.TitleBasics t
ON t.tconst = p.tconst
JOIN imdb_staging.NameBasics n
ON p.nconst = n.nconst

""


**check**

In [91]:
%%bigquery
SELECT * FROM datamart.TitlePrincipals LIMIT 5

,tconst,primaryTitle,nconst,primaryName,ordering,category,job,characters
0,tt11325996,Episode #1.1304,nm1532250,Ediberto Lima,2,director,\N,\N
1,tt4555146,Montgomery,nm0294975,Robert N. Fried,7,writer,\N,\N
2,tt3044534,Episode dated 24 July 2013,nm1202084,Jason Brandt,7,composer,\N,\N
3,tt10280490,Episode #1.32,nm8401844,Tiannan Cai,1,composer,\N,\N
4,tt1153542,Sara,nm0062197,Basia Baumann,5,director,\N,\N


#### **The primary key for this table is the combination of tconst and nconst. check for any records with the same primary key**

In [92]:
%%bigquery
SELECT tconst, nconst FROM datamart.TitlePrincipals GROUP BY tconst,nconst HAVING count(*) > 1 ORDER BY tconst

,tconst,nconst
0,tt0007388,nm0121364
1,tt0007702,nm0017629
2,tt0025628,nm0572762
3,tt0037841,nm0143419
4,tt0037961,nm0829725
...,...,...
1027,tt9839146,nm4381859
1028,tt9859758,nm9361072
1029,tt9871732,nm5149255
1030,tt9886982,nm9555390


In [93]:
%%bigquery
DELETE FROM datamart.TitlePrincipals a WHERE EXISTS (SELECT * FROM datamart.TitlePrincipals b WHERE a.tconst = b.tconst AND a.nconst = b.nconst AND a.characters = b.characters AND a.ordering != b.ordering )

""


In [94]:
%%bigquery
SELECT tconst, nconst FROM datamart.TitlePrincipals GROUP BY tconst,nconst HAVING count(*) > 1 ORDER BY tconst

,tconst,nconst
0,tt0037841,nm0143419
1,tt0108730,nm1470442
2,tt0195459,nm5766778
3,tt0280355,nm0222013
4,tt0428634,nm0391073
5,tt0466616,nm6730054
6,tt10115148,nm10587928
7,tt10461760,nm10053313
8,tt11448574,nm11033610
9,tt12650378,nm11713459


**The duplicates remaining are records where the same actor played multiple characters in the same title or have one record with a value for character and one record with a null value for character.** 

**Delete the duplicate with the less detailed or null value for character**

In [95]:
%%bigquery
DELETE FROM datamart.TitlePrincipals a WHERE EXISTS (SELECT * FROM datamart.TitlePrincipals b WHERE a.tconst = b.tconst AND a.nconst = b.nconst AND LENGTH(a.characters) < LENGTH(b.characters) AND a.ordering != b.ordering )

""


In [96]:
%%bigquery
SELECT tconst, nconst FROM datamart.TitlePrincipals GROUP BY tconst,nconst HAVING count(*) > 1 ORDER BY tconst

,tconst,nconst
0,tt0037841,nm0143419
1,tt0195459,nm5766778
2,tt11448574,nm11033610


**Fix the remaining three**

In [88]:
%%bigquery
SELECT * FROM datamart.TitlePrincipals WHERE nconst = "nm0143419" ORDER BY tconst

,tconst,primaryTitle,nconst,primaryName,ordering,category,job,characters
0,tt0037841,The Jury Goes Round 'n' Round,nm0143419,Elberta Hunter,9,actress,\N,"[""Tall Juror""]"
1,tt0037841,The Jury Goes Round 'n' Round,nm0143419,Elberta Hunter,10,actress,\N,"[""Tall juror""]"
2,tt1482408,California Girl,nm0143419,Elberta Hunter,1,actress,\N,\N


In [97]:
%%bigquery
DELETE FROM datamart.TitlePrincipals WHERE tconst = "tt0037841" AND nconst = "nm0143419" AND ordering = 10

""


In [98]:
%%bigquery
SELECT * FROM datamart.TitlePrincipals WHERE nconst = "nm5766778" ORDER BY tconst

,tconst,primaryTitle,nconst,primaryName,ordering,category,job,characters
0,tt0195459,The Fred Waring Show,nm5766778,Joe Marine,10,self,\N,"[""Self - Singer"",""Self""]"
1,tt0195459,The Fred Waring Show,nm5766778,Joe Marine,2,self,\N,"[""Self"",""Self - Singer""]"
2,tt11796756,"Bill Lawrence, Micki Marlo, Joe Marine",nm5766778,Joe Marine,3,self,\N,"[""Self - singer""]"
3,tt12090372,Richard Rodgers 25th Anniversary Celebration,nm5766778,Joe Marine,5,self,\N,"[""Self""]"
4,tt12217950,St. Patrick's Day Tribute,nm5766778,Joe Marine,7,self,\N,"[""Self""]"
5,tt12608366,Episode #3.21,nm5766778,Joe Marine,9,self,\N,"[""Self""]"
6,tt12817674,Spring Music,nm5766778,Joe Marine,7,self,\N,"[""Self""]"
7,tt13612876,"The Albins, Joe Marine, Harold & Lola",nm5766778,Joe Marine,7,self,\N,"[""Self - singer""]"
8,tt13953000,Episode #3.24,nm5766778,Joe Marine,7,self,\N,"[""Self""]"
9,tt14058132,April,nm5766778,Joe Marine,7,self,\N,"[""Self""]"


In [104]:
%%bigquery
DELETE FROM datamart.TitlePrincipals WHERE tconst = "tt0195459" AND nconst = "nm5766778" AND ordering = 10

""


In [100]:
%%bigquery
SELECT * FROM datamart.TitlePrincipals WHERE nconst = "nm11033610" ORDER BY tconst

,tconst,primaryTitle,nconst,primaryName,ordering,category,job,characters
0,tt11448574,Death Life,nm11033610,Taboo Acend,2,actor,\N,"[""Carl""]"
1,tt11448574,Death Life,nm11033610,Taboo Acend,5,actor,\N,"[""Jack""]"
2,tt11525700,The High School,nm11033610,Taboo Acend,1,actor,\N,"[""Leo Hart""]"
3,tt11535990,Welcome to Our High School,nm11033610,Taboo Acend,1,actor,\N,"[""Leo Hart""]"
4,tt11565178,Voting for School Captain,nm11033610,Taboo Acend,1,actor,\N,"[""Leo Hart""]"
5,tt11664394,Year 12 Excursion,nm11033610,Taboo Acend,1,actor,\N,"[""Leo Hart""]"
6,tt11670080,Honey,nm11033610,Taboo Acend,1,actor,\N,"[""Leo Hart""]"
7,tt11670112,March March,nm11033610,Taboo Acend,1,actor,\N,"[""Leo Hart""]"
8,tt11754918,School Musical Auditions,nm11033610,Taboo Acend,1,actor,\N,"[""Leo Hart""]"
9,tt11809986,The Gay Debate,nm11033610,Taboo Acend,1,actor,\N,"[""Leo Hart""]"


In [101]:
%%bigquery
DELETE FROM datamart.TitlePrincipals WHERE tconst = "tt11448574" AND nconst = "nm11033610" AND ordering = 5

""


**Check to see if any duplicates remain**

In [105]:
%%bigquery
SELECT tconst, nconst FROM datamart.TitlePrincipals GROUP BY tconst,nconst HAVING count(*) > 1 ORDER BY tconst

,tconst,nconst


## Add in Bollywood data

**These tables will be unable to be added into our main title/name tables until we are able to use apache to create new IDs for them in phase 3**

In [17]:
%%bigquery
CREATE TABLE datamart.BollywoodActors AS
SELECT Name, Height_in_cm_ 
FROM bollywood_staging.Actors

""


In [18]:
%%bigquery
CREATE TABLE datamart.BollywoodActresses AS
SELECT Name, Height_in_cm_, Debut_aslead_role
FROM bollywood_staging.Actress

""


In [134]:
%%bigquery
CREATE TABLE datamart.BollywoodTitle AS
SELECT *
FROM bollywood_staging.Bollywood

""


**delete the Highest_Grosser_By_Year_in_crores_ attribute in the BollywoodTitle table because it is 100% NULL**

In [135]:
%%bigquery
ALTER TABLE datamart.BollywoodTitle
DROP COLUMN Highest_Grosser_By_Year_in_crores_

""


**check**

In [25]:
%%bigquery
SELECT * FROM datamart.BollywoodActresses LIMIT 5

,Name,Height_in_cm_,Debut_aslead_role
0,Tanuja Samarth,152,Hamari Yaad Aayegi
1,Ameesha Patel,152,Kaho Naa... Pyaar Hai
2,Soha Ali Khan,152,Rang De Basanti
3,Farida Jalal,155,Laal Patthar
4,Waheeda Rehman,155,Pyaasa


**In its current state, the primary key of this table is Name. Check for any duplicates**

In [108]:
%%bigquery
SELECT count(*) as record_count FROM datamart.BollywoodActresses GROUP BY Name HAVING count(*) > 1 ORDER BY Name

,record_count


In [26]:
%%bigquery
SELECT * FROM datamart.BollywoodActors LIMIT 5

,Name,Height_in_cm_
0,Aamir Khan,163
1,Himesh Reshamiya,163
2,Kamal Haasan,165
3,Kunal Khemu,166
4,Shahid Kapoor,167


**In its current state, the primary key of this table is Name. Check for any duplicates**

In [136]:
%%bigquery
SELECT count(*) as record_count FROM datamart.BollywoodActors GROUP BY Name HAVING count(*) > 1 ORDER BY Name

,record_count


In [29]:
%%bigquery
SELECT * FROM datamart.BollywoodTitle LIMIT 5

,Year,Title,Director,Cast,Genre,Release_Month,Release_Date
0,1920,Shakuntala,Shree Nath Patankar,unknown,"mythology,drama",None,NaN
1,1921,Belgian Emperor's Visit To India,Nitin Bose,unknown,documentary,None,NaN
2,1921,King Gopichand a.k.a. Gopichand,Vishnupant Divekar,"mama,koregaonkar,hira,bhatt",legend,None,NaN
3,1921,Bilet Pherat a.k.a. England Returned,Dhirendranath Ganguly,"dhirendranath,chakraborty,sushilabala,kunjalal...","social,comedy",None,NaN
4,1922,Sharmista Devyani,Unknown,unknown,"drama,puranic",None,NaN


**In its current state, the primary key of this table is combination of Title, Director, and Year. Check for any duplicates**

In [137]:
%%bigquery
SELECT Title FROM datamart.BollywoodTitle GROUP BY Title, Year, Director HAVING count(*) > 1 ORDER BY Title

,Title
0,Rog


In [138]:
%%bigquery
SELECT * FROM datamart.BollywoodTitle WHERE Title = "Rog"

,Year,Title,Director,Cast,Genre,Release_Month,Release_Date
0,2005,Rog,Himanshu Brahmbhatt,"khan,irrfan,malik,himanshu,ilene,hamann","drama,thriller",None,NaN
1,2005,Rog,Himanshu Brahmbhatt,unknown,"drama,thriller",None,NaN


**Delete the duplicate that has less information on Cast**

In [1]:
%%bigquery
DELETE FROM datamart.BollywoodTitle WHERE Title = "Rog" AND Cast = "unknown" AND Year = "2005"

Executing query with job ID: 0fb66391-9339-4b23-b071-70db29dfa850
Query executing: 0.96s


ERROR:
 400 Syntax error: Expected "(" but got "=" at [1:66]

(job ID: 0fb66391-9339-4b23-b071-70db29dfa850)

                                  -----Query Job SQL Follows-----                                  

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:DELETE FROM datamart.BollywoodTitle WHERE Title = "Rog" AND Cast = "unknown" AND Year = "2005"
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |


**We tried to delete the duplicate but we are getting this error for some reason. We will try to fix this in Phase 3**

## Write Queries

**Get the highest rated film with the highest number of votes for each collection of genres, order by the votes in descending order**

In [153]:
%%bigquery
SELECT primaryTitle, genres, averageRating, numVotes 
FROM datamart.TitleBasics t 
WHERE averageRating = (SELECT max(averageRating) FROM datamart.TitleBasics a WHERE a.genres = t.genres) 
AND numVotes = (SELECT max(numVotes) FROM datamart.TitleBasics b WHERE b.genres = t.genres) 
ORDER BY numVotes DESC LIMIT 10

,primaryTitle,genres,averageRating,numVotes
0,Inglourious Basterds,"Adventure,Drama,War",8.3,1287439
1,The Big Lebowski,"Comedy,Crime,Sport",8.1,740570
2,Vertigo,"Mystery,Romance,Thriller",8.3,369005
3,Grave of the Fireflies,"Animation,Drama,War",8.5,240211
4,Sunset Blvd.,"Drama,Film-Noir",8.4,204110
5,The Third Man,"Film-Noir,Mystery,Thriller",8.1,160523
6,The Maltese Falcon,"Crime,Film-Noir,Mystery",8.0,150049
7,Double Indemnity,"Crime,Drama,Film-Noir",8.3,145328
8,Lagaan: Once Upon a Time in India,"Drama,Musical,Sport",8.1,105953
9,Notorious,"Drama,Film-Noir,Romance",7.9,93094


**Find titles in the "Comedy,Horror" genre which have above average ratings for the genre, order by number of votes in descending order**

In [152]:
%%bigquery
SELECT primaryTitle, averageRating, numVotes
FROM datamart.TitleBasics t
WHERE averageRating > (SELECT avg(averageRating) FROM datamart.TitleBasics a WHERE a.genres = t.genres) AND genres = "Comedy,Horror" ORDER BY numVotes DESC LIMIT 10

,primaryTitle,averageRating,numVotes
0,Shaun of the Dead,7.9,516784
1,Tucker and Dale vs Evil,7.5,166564
2,Army of Darkness,7.5,163921
3,What We Do in the Shadows,7.7,160157
4,Evil Dead II,7.8,149553
5,The Lost Boys,7.3,125707
6,Tremors,7.1,121812
7,Fright Night,6.4,100280
8,An American Werewolf in London,7.5,94259
9,Trick 'r Treat,6.8,81640


**Find the 5 shows with the highest amount of episodes**

In [161]:
%%bigquery
SELECT a.episode_count, b.primaryTitle FROM (SELECT COUNT(*) as episode_count, parentTconst FROM datamart.TitleEpisode GROUP BY parentTconst) a JOIN datamart.TitleBasics b ON a.parentTconst = b.tconst ORDER BY episode_count DESC LIMIT 5

,episode_count,primaryTitle
0,14269,Days of Our Lives
1,12277,The Young and the Restless
2,10204,Coronation Street
3,9958,The Tonight Show Starring Johnny Carson
4,9858,Charlie Rose


**Find the 5 titles with the most alternate regional titles**

In [173]:
%%bigquery
SELECT a.title_count, b.primaryTitle FROM (SELECT count(*) as title_count, titleId FROM datamart.TitleAkas 
GROUP BY titleId) a JOIN datamart.TitleBasics b ON a.titleId = b.tconst ORDER BY a.title_count DESC LIMIT 5

,title_count,primaryTitle
0,159,Pokémon
1,106,Star Wars: Episode IV - A New Hope
2,98,The Hollow Forest Chronicles
3,89,Superhero Landing
4,88,Sailor Moon


**Find the 5 highest rated genres**

In [184]:
%%bigquery
select genres, AVG(averageRating) as average_rating FROM datamart.TitleBasics GROUP BY genres ORDER BY AVG(averageRating) DESC LIMIT 5

,genres,average_rating
0,"Comedy,History,Western",9.800000
1,"News,Reality-TV,Short",9.700000
2,"Adventure,Music,Mystery",9.621429
3,"Music,Musical,Talk-Show",9.416867
4,"Biography,Comedy,Sport",9.400000


## Create new dataset and views

In [3]:
!bq --location=US mk --dataset {"reports"}

Dataset 'authentic-light-303018:reports' successfully created.


**Create a view for the query: " get the genres with the highest average ratings"**

**Modify the query to LIMIT 25**

In [11]:
%%bigquery
CREATE VIEW reports.Highest_Average_Rated_Genres AS
select genres, AVG(averageRating) as average_rating FROM datamart.TitleBasics GROUP BY genres ORDER BY AVG(averageRating) DESC LIMIT 25

""


**check**

In [12]:
%%bigquery
SELECT * FROM reports.Highest_Average_Rated_Genres LIMIT 5

,genres,average_rating
0,"Comedy,History,Western",9.800000
1,"News,Reality-TV,Short",9.700000
2,"Adventure,Music,Mystery",9.621429
3,"Music,Musical,Talk-Show",9.416867
4,"Biography,Comedy,Sport",9.400000


**Create view for the query: "Get the shows with the most episodes"**

**Modify the query to LIMIT 50**

In [7]:
%%bigquery
CREATE VIEW reports.Shows_With_Most_Episodes AS
SELECT a.episode_count, b.primaryTitle FROM (SELECT COUNT(*) as episode_count, parentTconst FROM datamart.TitleEpisode GROUP BY parentTconst) a JOIN datamart.TitleBasics b ON a.parentTconst = b.tconst ORDER BY episode_count DESC LIMIT 50

""


**Check**

In [9]:
%%bigquery
SELECT * FROM reports.Shows_With_Most_Episodes LIMIT 5

,episode_count,primaryTitle
0,14269,Days of Our Lives
1,12277,The Young and the Restless
2,10204,Coronation Street
3,9958,The Tonight Show Starring Johnny Carson
4,9858,Charlie Rose
